## Saving And Loading A general Checkpoint in Pytorch

- 추론을 위해 일반 체크포인트 모델을 저장하고 로드하거나 교육을 다시 시작하는 것은 마지막으로 중단한 부분을 선택하는 데 도움이 될 수 있습니다
- 일반 체크포인트를 저장할 때는 모델의 state_dict 이상의 체크포인트를 저장해야 합니다.
- 또한 optimizer의 state_dict를 저장하는 것이 중요합니다. 여기에는 모델 train로 업데이트되는 버퍼 및 매개 변수가 포함되어 있기 때문입니다.
- 저장하고자 하는 다른 항목은 중단한 시점, 가장 최근에 기록된 train loss, 외부 torch.nn입니다.자체 알고리즘에 따라 레이어 등을 내장합니다.
- https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html

#### Introduction

- 여러 체크포인트를 저장하려면 사전으로 구성하고 torch.save()를 사용하여 사전을 직렬화해야 합니다. 일반적인 PyTorch 규칙은 .tar 파일 확장자를 사용하여 이러한 체크포인트를 저장하는 것입니다.
- 항목을 로드하려면 먼저 모델 및 최적화 프로그램을 초기화한 다음 토치.load()를 사용하여 사전을 로컬로 로드하십시오.
- 여기에서 예상한 대로 사전을 조회하기만 하면 저장된 항목에 쉽게 액세스할 수 있습니다.

Steps
- 1. 데이터를 로드하는 데 필요한 모든 라이브러리 가져오기
- 2. 신경망을 정의하고 초기화합니다.
- 3. 최적화 도구 초기화
- 4. 일반 체크포인트 저장
- 5. 일반 체크포인트 로드

#### 1. 데이터를 로드하는 데 필요한 모든 라이브러리 가져오기

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

#### 2. 신경망을 정의하고 초기화합니다.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### 3. 최적화 도구 초기화

In [4]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
net.parameters()

<generator object Module.parameters at 0x7fda64ec8850>

#### 4. 일반 체크포인트 저장

In [6]:
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch':EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
    }, PATH
    )
    

#### 5. 일반 체크포인트 로드

In [7]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
los = checkpoint['loss']

model.eval()
model.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)